In [1]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import display
from loguru import logger

# from solhycool_visualizations import save_figure
# from solhycool_visualizations.constants import generate_plotly_config

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# reload
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

filename_opt_result = '20240108_optimization_results.json'
filename_process_data = '20240108_process_timeseries.csv'

sample_rate = '60s'

initial_datetime = '2024-01-08 10:55'
final_datetime = '2024-01-08 14:00'

active_idxs = [1, 3, 5, 7]

# Pre-processing

In [45]:
# Load system information and data
with open(data_path / 'config.json') as f:
    config = json.load(f)
    
with open(results_path / filename_opt_result) as f:
    opt_results = json.load(f)

logger.info(f'Number of evaluations prior to selection: {len(opt_results)}')

# Filter out the results that are not active
# Convert the ordered dictionary to a list of key-value pairs
items_to_delete = list(opt_results.items())
# Indexes not in active_idxs and [0...len(opt_results)]
opt_results_list = [i for i in range(len(opt_results)) if i not in active_idxs]
# Iterate over the indexes in reverse order and delete the corresponding items
for index in sorted(opt_results_list, reverse=True):
    del items_to_delete[index]
# Create a new ordered dictionary with the remaining items
opt_results = OrderedDict(items_to_delete)

with open(src_path / "visualizations" / "plot_config.hjson") as f:
    plt_config = hjson.load(f)
    
# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(results_path / filename_process_data, parse_dates=True, index_col='time')

# Set UTC timezone
df = df.tz_localize('UTC')

logger.info(f'Number of evaluations after selection: {len(opt_results)}')

2024-01-10 20:28:31.013 | INFO     | __main__:<module>:8 - Number of evaluations prior to selection: 11
2024-01-10 20:28:31.502 | INFO     | __main__:<module>:30 - Number of evaluations after selection: 4


In [46]:
# Sample every 60 seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

In [47]:
# Add missing variables, this should be done in the optimization code
# opt_results[ list(opt_results.keys())[0] ]['selected_solution_idx'] = 3

from iapws import IAPWS97

# TODO: Add Pth

# for idx, row in df.iterrows():
#     cp = IAPWS97(P=2, T=row.mean() + 273.15).cp
df['Pth'] = 4.178 * df['q_c']/3.6 * (df['Tc_out'] - df['Tc_in'])

# Cambiar valor de Pth para optimziatión de las 13:30
cs_id = 'CS_20240108_13H30M_Ta14_HR39_Tv45_Pth195'
opt_results[cs_id]['cooling_requirements']['Pth'] = df.loc['2024-01-08 13:25':'2024-01-08 13:30']['Pth'].mean()

logger.warning(f"Manually change valued of Pth for case study CS_20240108_13H30M_Ta14_HR39_Tv45_Pth195 to {opt_results[cs_id]['cooling_requirements']['Pth']:.2f} kW")

2024-01-10 20:28:31.617 | WARNING  | __main__:<module>:16 - Manually change valued of Pth for case study CS_20240108_13H30M_Ta14_HR39_Tv45_Pth195 to 160.01 kW


In [48]:
# From each dictionary in opt_results, append a row to the dataframe with:
# time, computation_time, and the values of solutions[selected_solution_idx+1]

temp_data = []

for i, opt_result in enumerate(opt_results.values()):
    solution_obj = opt_result['solutions'][opt_result['selected_solution_idx'] - 1] # MATLAB index starts at 1
    solution_obj['time'] = opt_result['time']
    solution_obj['computation_time'] = opt_result['computation_time']
    
    temp_data.append(solution_obj)
    
temp_df = pd.DataFrame(temp_data)


# First convert the time field to datetime
# temp_df = pd.DataFrame(opt_results)
temp_df['time'] = pd.to_datetime(temp_df['time'])
temp_df.set_index('time', inplace=True)
# temp_df = temp_df.tz_localize('UTC')

continuous_time_index = pd.date_range(start=temp_df.index.min(), end=df.index.max(), freq='1S')
df_opt = temp_df.reindex(continuous_time_index)
df_opt.ffill(inplace=True)
df_opt = df_opt.resample(sample_rate).mean()

In [49]:
# Combine the optimization results with the timeseries data
# From optimization result, build a dataframe with the same index as df, by concatenating the values using the time field

# Add suffix to colums and join with df
df_opt = df_opt.add_suffix('_opt')

df = df.join(df_opt, how='outer')

# Return to original index to be used later in the plot
df_opt = temp_df

In [50]:
# Filter out unreasonable consumption values

df['Ce_opt'].where(df['Ce_opt'] < 100, np.nan, inplace=True)
df['Cw_opt'].where(df['Cw_opt'] < 1000, np.nan, inplace=True)
df_opt['Ce'].where(df_opt['Ce'] < 100, np.nan, inplace=True)
df_opt['Cw'].where(df_opt['Cw'] < 1000, np.nan, inplace=True)

# Calculate active states for DC and WCT, this should be read from the timeseries data
df['dc_active'] = df['w_dc'] > 11
df['wct_active'] = df['w_wct'] > 21

In [51]:
# Calculate additional variables
from solhycool_visualizations.calculations import power_consumption

# df["C_e"] = consumption_fit(df["pump flow or freq"]) ...
# df["C"] = df["C_e"] * lambda_e + df["C_w"] * lambda_w

df["Ce_dc"] = power_consumption(df["w_dc"].to_numpy(), actuator='fan_dc')
df["Ce_wct"] = power_consumption(df["w_wct"].to_numpy(), actuator='fan_wct')
df["Ce_c"] = power_consumption(df["q_c"].to_numpy(), actuator='pump_c')

df["Ce"] = df["Ce_dc"] + df["Ce_wct"] + df["Ce_c"] 
df["Cw"] = df["Cw"] * 60

In [52]:
# Cut the dataframe to the desired time range

# df = df[:-19*60]
# Remove until 10:30

df = df[initial_datetime:final_datetime]
logger.info(f'Trimmed dataframe from {df.index[0]} to {df.index[-1]}')

2024-01-10 20:28:31.969 | INFO     | __main__:<module>:7 - Trimmed dataframe from 2024-01-08 10:55:00+00:00 to 2024-01-08 14:00:00+00:00


# Visualizations

## Frente de pareto
Frentes de pareto acumulados hasta ahora para los distintos casos de estudio, punto seleccionado se muestra resaltado

In [53]:
%autoreload 2

from solhycool_visualizations.pareto import pareto_plot

fig = pareto_plot(opt_results, Cws_max=(125, 150), xlim=(50, 190), ylim=(-0.5, 5), full_legend=True)

fig.show(config=generate_plotly_config(fig))

In [54]:
save_figure(
    figure_name=f"pareto_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=results_path / "figures", 
    fig=fig, formats=['png', 'svg', 'html'], 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

2024-01-10 20:28:47.314 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/pareto_20240108.png
2024-01-10 20:28:47.482 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/pareto_20240108.svg
2024-01-10 20:28:47.490 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/pareto_20240108.html


## Evolución temporal de consumos
Comparación de los consumos de consumos entre la evolución de la serie temporal y los estimados por el modelo 

In [15]:
from solhycool_visualizations.costs import costs_evolution_plot

fig = costs_plot(df)

fig.show()

NameError: name 'costs_plot' is not defined

In [ ]:
save_figure(
    figure_name=f"costs_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=results_path / "figures", 
    fig=fig, formats=['png', 'svg', 'html'], 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

## Resultados experimentales
Gráfica del sistema completo mostrando comparativa entre las series temporales obtenidas experimentalmente y las estimadas por el modelo

In [2]:
# from solhycool_visualizations.experimental_results import experimental_results_plot
from phd_visualizations.test_timeseries import experimental_results_plot

# Load plot configuration here as well to not have to go back and forth
with open(src_path / "visualizations" / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, df_opt)

fig.show()

NameError: name 'df' is not defined

In [ ]:
save_figure(
    figure_name=f"experimental_results_{df.index[0].strftime('%Y%m%d')}",
    figure_path=results_path / "figures",
    fig=fig, formats=['png', 'svg', 'html'],
    width=fig.layout.width, height=fig.layout.height, scale=2
)

## Distribución de caudales
Distribución de caudales en el sistema

In [52]:
import plotly.graph_objects as go
from solhycool_visualizations.constants import color_palette, default_fontsize

case_study = list(opt_results.values())[0]
p = case_study['solutions'][ case_study['selected_solution_idx']+1 ]

R1 = p['R1']
R2 = p['R2']
dc = 1-R1
wct_pp = R1
just_dc = dc*(1-R2)
dc_wct = dc*R2
wct = wct_pp + dc_wct

N = 5
alpha = 0.6
cp = color_palette
dc_color = f'rgba({cp["dc_green_rgb"]}, {alpha})'
wct_color = f'rgba({cp["wct_purple_rgb"]}, {alpha})'
c_color = f'rgba({cp["c_blue_rgb"]}, {alpha})'

fig = go.Figure(
    data=[
        go.Sankey(
            
            arrangement = "freeform",
            node = dict(
                pad = 15,
                thickness = 20,
                
                line = dict(color = color_palette["bg_blue"], width = 0.5),
                label = ["q<sub>c</sub>", "DC", "WCT PP", "DC-WCT", "just DC", "WCT", "q<sub>c</sub>"],
                #      [0                 1      2         3          4        5        6]
                color = color_palette["bg_blue"],
                x = [0.1, 0.3, 0.3, 0.5, 0.5, 0.7, 0.9 ],
                # y = [0.5, 0.7, 0.3, 0.3, 0.7, 0.3, 0.5]
            ),
            link = dict(
                arrowlen=1,
                source = [0,  0,      1,        4,      1,      2,      3,         5],
                target = [1,  2,      4,        0,      3,      5,      5,         0],
                value  = [dc, wct_pp, just_dc, just_dc, dc_wct, wct_pp, dc_wct, wct],
                color = [dc_color, wct_color, dc_color, c_color, wct_color, wct_color, wct_color, c_color]
            )
        )
    ]
)

fig.update_layout(
    title_text="Basic Sankey Diagram", 
    font_size=default_fontsize,
    height=600,
    width=1000,
)

fig.show(config=generate_plotly_config(fig))

In [54]:
save_figure(
    figure_name=f"sankey_{case_study['id']}",
    figure_path=results_path / "figures",
    fig=fig, formats=['png', 'svg', 'html'],
    width=fig.layout.width, height=fig.layout.height, scale=2
)

2024-01-10 13:40:26.927 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/sankey_CS_20240108_11H12M_Ta12_HR48_Tv45_Pth165.png
2024-01-10 13:40:26.950 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/sankey_CS_20240108_11H12M_Ta12_HR48_Tv45_Pth165.svg
2024-01-10 13:40:26.952 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/sankey_CS_20240108_11H12M_Ta12_HR48_Tv45_Pth165.html


## Visualización de las distintas configuraciones escogidas

In [71]:
print(df_opt.index)

DatetimeIndex(['2024-01-08 11:12:46+00:00', '2024-01-08 11:45:03+00:00',
               '2024-01-08 12:17:21+00:00', '2024-01-08 13:30:53+00:00'],
              dtype='datetime64[ns, UTC]', name='time', freq=None)


In [91]:
import plotly.graph_objs as go
from solhycool_visualizations.constants import color_palette, default_fontsize

display(df_opt.head())

PP = df_opt['R1'] * 100
SS = df_opt['q_vm'] / df_opt['q_c'] * 100
just_DC = (1 - df_opt['R1']) * (1 - df_opt['R2']) * 100

# Create a figure where the xaxis is a categorical axis with the list of case studies, each case study will have two bards, one for PP and one for SS, the height of the bars will be the values of PP and SS

xvalues = np.arange(len(df_opt.index))

fig = go.Figure(
    data=[
        go.Bar(
            name='Parallel',
            x=xvalues, y=PP,
            # marker_color=color_palette['dc_green'],
            # marker_line_color=color_palette['dc_green'],
            # marker_line_width=1.5,
            opacity=0.6
        ),
        go.Bar(
            name='Series DC - WCT',
            x=xvalues, y=SS,
            # marker_color=color_palette['wct_purple'],
            # marker_line_color=color_palette['wct_purple'],
            # marker_line_width=1.5,
            opacity=0.6
        ),
        go.Bar(
            name='DC only',
            x=xvalues, y=just_DC,
            # marker_color=color_palette['wct_purple'],
            # marker_line_color=color_palette['wct_purple'],
            # marker_line_width=1.5,
            opacity=0.6
        )
    ]
)

# Set marks of the xaxis from df_opt.index, space evenly
fig.update_xaxes(
    tickvals=xvalues,
    ticktext=[idx.strftime("%H:%M") for idx in df_opt.index],
)

fig.update_layout(
    title_text="<b>Operation strategy</b><br> ",
    font_size=default_fontsize,
    height=600,
    width=800,
    xaxis_title="Case study",
    yaxis_title="Percentage",
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    plot_bgcolor='rgba(255,255,255,0.5)',
    legend=dict(
        title_text="Configuration",
        orientation="h",
        yanchor="bottom",
        y=0.98,
        xanchor="right",
        x=1
    )
)

fig.show(config=generate_plotly_config(fig))

,Tamb,HR,R1,R2,Ce_dc,Ce_wct,Ce_cc,Cw_wct,q_wct,q_dc,...,Pth,Tc_in,Ce,Cw,Ce_c,q_c_min,q_c_max,q_dc_min,q_wct_min,computation_time
time,,,,,,,,,,,,,,,,,,,,,
2024-01-08 11:12:46+00:00,12.235008,48.199804,0.482368,0.950691,0.052505,0.130985,0.183489,142.769279,15.343790,8.150460,...,164.519621,31.966030,1.807055,142.769279,1.623566,12,24,5,5.7,313.322482
2024-01-08 11:45:03+00:00,12.599754,47.014233,0.381558,0.859442,0.057333,0.141316,0.198650,144.419823,16.767798,11.357141,...,189.653332,31.964161,2.571076,144.419823,2.372426,12,24,5,5.7,419.036150
2024-01-08 12:17:21+00:00,13.187537,46.045497,0.351124,0.894782,0.101954,0.132520,0.234474,149.261974,15.248392,10.619331,...,173.792500,32.539397,2.019095,149.261974,1.784620,12,24,5,5.7,325.013282
2024-01-08 13:30:53+00:00,14.071567,39.171157,0.180487,0.940048,0.071204,0.130187,0.201391,123.907855,17.727388,15.278481,...,194.945777,31.948542,2.664576,123.907855,2.463185,12,24,5,5.7,338.504662


In [92]:
save_figure(
    figure_name=f"operation_strategies_{df_opt.index[0].strftime('%Y%m%d')}",
    figure_path=results_path / "figures",
    fig=fig, formats=['png', 'svg', 'html'],
    width=fig.layout.width, height=fig.layout.height, scale=2
)

2024-01-10 21:12:38.278 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/operation_strategies_20240108.png
2024-01-10 21:12:38.343 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/operation_strategies_20240108.svg
2024-01-10 21:12:38.346 | INFO     | solhycool_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024/results/figures/operation_strategies_20240108.html


## Diagrama de la instalación
Diagrama de la instalación para visualizar el punto de operación estimado por el modelo

In [ ]:
from solhycool_visualizations.diagrams import generate_facility_diagram
from IPython.display import Image
import subprocess

src_diagram_path = f"{os.getenv('HOME')}/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202309_JJA23/wascop_app/assets/optimization_V1/aux/WASCOP-Resultados JJAA.svg"

for case_study_id in opt_results:
    case_study = opt_results[case_study_id] 
    
    # Manual variable renaming, suposed to be fixed for future versions
    # TODO: Remove this when fixed in the optimization code (get_bounds.m)
    case_study['limits']['w_dc_min'] = case_study['limits']['w_fan_min']
    case_study['limits']['w_wct_min'] = case_study['limits']['w_fan_min']
    case_study['limits']['w_dc_max'] = case_study['limits']['w_fan_dc_max']
    case_study['limits']['w_wct_max'] = case_study['limits']['w_fan_wct_max']
    
    output_path = results_path / "figures"
    filename = f"{case_study['time'].strftime('%Y%m%dT%HH%MM')}_facility_diagram"
    
    generate_facility_diagram(src_diagram_path, case_study, save_diagram=True, output_path=output_path / f'{filename}.svg')
    
    # Convert to png using inkscape and subprocess
    subprocess.run(
        [
            "inkscape", 
            '--export-type=png', 
            '--export-dpi=200', 
            output_path / f'{filename}.svg'
        ]
    )
    
    display( Image(output_path / f'{filename}.png') )

In [ ]:
# For one of the case studies, generate diagrams for all pareto points to visualize how the operation strategy changes

for case_study_id in opt_results:

    # # Choose one case study
    # case_study = opt_results[ list(opt_results.keys())[1] ]
    case_study = opt_results[case_study_id]
    
    # Create directory for the diagrams
    os.makedirs(results_path / "figures" / case_study_id, exist_ok=True)
    
    # Manual variable renaming, suposed to be fixed for future versions
    # TODO: Remove this when fixed in the optimization code (get_bounds.m)
    case_study['limits']['w_dc_min'] = case_study['limits']['w_fan_min']
    case_study['limits']['w_wct_min'] = case_study['limits']['w_fan_min']
    case_study['limits']['w_dc_max'] = case_study['limits']['w_fan_dc_max']
    case_study['limits']['w_wct_max'] = case_study['limits']['w_fan_wct_max']
    
    for idx, ptop in enumerate(case_study['solutions']):
        
        output_path = results_path / f"figures/{case_study_id}"
        filename = f"cs_{case_study['id']}_ptop_{idx}_facility_diagram"
        
        case_study['selected_solution_idx'] = idx + 1
        
        generate_facility_diagram(src_diagram_path, case_study, save_diagram=True, output_path=output_path / f'{filename}.svg')
        
        # Convert to png using inkscape and subprocess
        subprocess.run(
            [
                "inkscape", 
                '--export-type=png', 
                '--export-dpi=200', 
                output_path / f'{filename}.svg'
            ]
        )